In [ ]:
import os
import subprocess
os.chdir('/Users/aaron/Desktop/help/natasha')

In [12]:
[subprocess.run('mv "{}" {}'.format(x, x.split(' ')[0] + '.raw'), shell=True) for x in os.listdir() if '(' in x]

[CompletedProcess(args='mv "ASM_1 (1).raw" ASM_1.raw', returncode=0),
 CompletedProcess(args='mv "MEB_1 (5).raw" MEB_1.raw', returncode=0),
 CompletedProcess(args='mv "LB_C (1).raw" LB_C.raw', returncode=0),
 CompletedProcess(args='mv "ASM_C (1).raw" ASM_C.raw', returncode=0),
 CompletedProcess(args='mv "LB_1 (1).raw" LB_1.raw', returncode=0),
 CompletedProcess(args='mv "MEB_C (1).raw" MEB_C.raw', returncode=0)]

In [5]:
raws = [os.path.abspath(f'raw/{x}') for x in os.listdir('raw') if '.raw' in x]

with open('run_ThermoRawFileParser.sh', 'w') as f:
    for raw in raws:
        cmd = f'mono ThermoRawFileParser1.4.5/ThermoRawFileParser.exe -i={raw} -o mzml\n'
        f.write(cmd)



In [ ]:
import os
import pandas as pd
os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')

In [ ]:
if not os.path.exists('ska_within_host'):
    os.makedirs('ska_within_host')

In [ ]:
df = pd.read_csv('data/metadata.tsv', sep='\t')
patients = open('data/patients.txt', 'r').read().splitlines()


['P01', 'P03', 'P05', 'P07', 'P08', 'P09', 'P11', 'P12', 'P13', 'P14', 'P15', 'P17', 'P18', 'P20', 'VT_070', 'VT_086', 'VT_104', 'VT_124', 'VT_169', 'VT_199', 'VT_218', 'VT_221', 'VT_260', 'VT_261', 'VT_267', 'VT_275', 'Wetzstein_007', 'Wetzstein_024', 'Wetzstein_046', 'Wetzstein_047', 'Wetzstein_060', 'Wetzstein_065', 'Wetzstein_090', 'Wetzstein_121', 'Wetzstein_125', 'Wetzstein_132', 'Wetzstein_144', 'Wetzstein_151', 'Wetzstein_176', 'Wetzstein_194']


In [ ]:
ska_build = 'ska build -f {out_dir}/file_list.txt -k 31 -o {out_dir}/index --threads 8 -v'
ska_align = 'ska align --min-freq 1 --filter no-filter {out_dir}/index.skf -o {out_dir}/alignment.aln --ambig-mask --threads 8 -v'
snp_sites = 'docker run -v $PWD:/data staphb/snp-sites snp-sites -c {out_dir}/alignment.aln > {out_dir}/alignment.core.aln'
snp_dists = 'docker run -v $PWD:/data staphb/snp-dists snp-dists {out_dir}/alignment.core.aln > {out_dir}/snp_dists.txt'

with open('run_ska_within_host.sh', 'w') as f:
    for patient in patients:
        out_dir = f'ska_within_host/{patient}'
        for cmd in [ska_build, ska_align, snp_dists, snp_sites]:
            f.write(cmd.format(out_dir=out_dir) + '\n')
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        isolates = df[df['patient'] == patient]['isolate'].tolist()
        file_list = f'{out_dir}/file_list.txt'
        with open(file_list, 'w') as filelist:
            for isolate in isolates:
                reads = '\t'.join([f'reads/processed/{isolate}_R{x}_001.qcd.fastq.gz' for x in [1, 2]])
                filelist.write(f'{isolate}\t{reads}\n')


In [14]:
snp_sites = 'docker run -v $PWD:/data staphb/snp-sites snp-sites -c {out_dir}/alignment.aln > {out_dir}/alignment.core.aln'
snp_dists = 'docker run -v $PWD:/data staphb/snp-dists snp-dists {out_dir}/alignment.core.aln > {out_dir}/snp_dists.txt'

with open('run_ska_within_host_snp_dists.sh', 'w') as f:
    for patient in patients:
        out_dir = f'ska_within_host/{patient}'
        for cmd in [snp_sites, snp_dists]:
            f.write(cmd.format(out_dir=out_dir) + '\n')